In [25]:
from mantle import mantle
from env import *
from json import dumps
from ipynb.fs.defs.common_last_synced_height import last_synced_height

def query_collateral_list(wallet_addr: str):
  block_height = last_synced_height()

  return mantle(
    wasm_query={
      'overseer_whitelist': {
        'contractAddress': moneymarket_overseer_addr,
        'query': {
          'whitelist': {}
        }
      },
      'oracle_prices': {
        'contractAddress': moneymarket_oracle_addr,
        'query': {
          'prices': {}
        }
      },
      'market_borrower_info': {
        'contractAddress': moneymarket_market_addr,
        'query': {
          'borrower_info': {
            'borrower': wallet_addr,
            'block_height': block_height
          }
        }
      },
      'overseer_collaterals': {
        'contractAddress': moneymarket_overseer_addr,
        'query': {
          'collaterals': {
            'borrower': wallet_addr
          }
        }
      },
      'overseer_borrow_limit': {
        'contractAddress': moneymarket_overseer_addr,
        'query': {
          'borrow_limit': {
            'borrower': wallet_addr,
            'block_height': block_height
          }
        }
      }
    }
  )

print(dumps(query_collateral_list('terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9'), indent=2))

{
  "market_borrower_info": {
    "borrower": "terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9",
    "interest_index": "1.065749573609246711",
    "loan_amount": "68792519",
    "pending_rewards": "863932.585190313116911126",
    "reward_index": "708.221073540025979306"
  },
  "oracle_prices": {
    "Ok": {
      "prices": [
        {
          "asset": "terra19mkj9nec6e3y5754tlnuz4vem7lzh4n0lc2s3l",
          "last_updated_time": 1629013143,
          "price": "3208.91586809"
        },
        {
          "asset": "terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x",
          "last_updated_time": 1629013159,
          "price": "16.776045290335027136"
        }
      ]
    }
  },
  "overseer_borrow_limit": {
    "borrow_limit": "108426270",
    "borrower": "terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9"
  },
  "overseer_collaterals": {
    "borrower": "terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9",
    "collaterals": [
      [
        "terra19mkj9nec6e3y5754tlnuz4vem7lzh4n0lc2s3l",
        "11

In [29]:
result = query_collateral_list('terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9')

if result['oracle_prices']['Ok'] is not None:
  oracle_prices = result['oracle_prices']['Ok']['prices']
else:
  oracle_prices = result['oracle_prices']['prices']

whiltelist_elems = result['overseer_whitelist']['elems']
user_collaterals = result['overseer_collaterals']['collaterals']
loan_amount = int(result['market_borrower_info']['loan_amount'])

for whitelist in whiltelist_elems:
  token_addr = whitelist['collateral_token']
  price = next((item for item in oracle_prices if item['asset'] == token_addr), None)
  locked_amount = next((item for item in user_collaterals if item[0] == token_addr), None)

  print('symbol', whitelist['symbol'])
  print('name', whitelist['name'])
  print('token_addr', token_addr)
  print('price (in UST)', price['price'])
  print('provided', int(locked_amount[1]) / 1000000)
  print('provided (in UST)', int(locked_amount[1]) * float(price['price']) / 1000000)

  # only possible calculate the liquidation_price when the collateral have 1
  if len(user_collaterals) == 1:
    collateral = next((item for item in user_collaterals if item[0] == token_addr), None)
    max_ltv = float(whitelist['max_ltv'])
    liquidation_price = loan_amount / int(collateral[1]) * max_ltv
    print('max_ltv', max_ltv)
    print('liquidation_price', liquidation_price)

  print('')

symbol BETH
name Bonded ETH
token_addr terra19mkj9nec6e3y5754tlnuz4vem7lzh4n0lc2s3l
price (in UST) 3208.91586809
provided 0.01135
provided (in UST) 36.4211951028215

symbol BLUNA
name Bonded Luna
token_addr terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x
price (in UST) 16.721120313049809825
provided 8.60091
provided (in UST) 143.81685091171323

